In [42]:
!pip install gradio

In [43]:
!pip install xlsxwriter

In [44]:
import pandas as pd
import random
import gradio as gr
import ast
import io
import contextlib
import os
import shutil
from io import BytesIO

In [45]:
def process_excel_files(file1, file2, file3):
    df = pd.read_excel(file1.name)

    prof_course_capacity = []
    cross_listed_courses = {}

    professors = df.columns[3:27]

    undergrad_courses = df.iloc[1:30, 0]
    for i, course in enumerate(undergrad_courses):
        for j, professor in enumerate(professors):
            cell_value = df.iloc[i+1, j+3]
            if cell_value == 1 or cell_value == 'x':
                capacity = df.iloc[i+1, 27]
                prof_course_capacity.append((professor, course, capacity))
                if cell_value == 'x':
                    if professor not in cross_listed_courses:
                        cross_listed_courses[professor] = [course]
                    else:
                        cross_listed_courses[professor].append(course)

    grad_courses = df.iloc[31:52, 0]
    for i, course in enumerate(grad_courses):
        for j, professor in enumerate(professors):
            cell_value = df.iloc[i+31, j+3]
            if cell_value == 1 or cell_value == 'x':
                capacity = df.iloc[i+31, 27]
                prof_course_capacity.append((professor, course, capacity))
                if cell_value == 'x':
                    if professor not in cross_listed_courses:
                        cross_listed_courses[professor] = [course]
                    else:
                        cross_listed_courses[professor].append(course)

    # Modify 'prof_course_capacity' to combine cross-listed courses
    for professor, courses in cross_listed_courses.items():
        combined_course = '+'.join(courses)
        for course in courses:
            for i, (prof, old_course, capacity) in enumerate(prof_course_capacity):
                if prof == professor and old_course == course:
                    prof_course_capacity[i] = (professor, combined_course, capacity)

    # Remove duplicates
    prof_course_capacity = list(set(prof_course_capacity))

    course_capacity_dict = {}
    for professor, course, capacity in prof_course_capacity:
        course_capacity_dict[course] = capacity


    # First group the professor+course by professor
    pcc_grouped = {}
    for pcc in prof_course_capacity:
        professor, course, capacity = pcc
        if professor not in pcc_grouped:
            pcc_grouped[professor] = [(course, capacity)]
        else:
            pcc_grouped[professor].append((course, capacity))

    # Modify undergrad_courses and Modify grad_courses
    for professor, courses in cross_listed_courses.items():
        for course in courses:
            if course in undergrad_courses.values:
                undergrad_courses = undergrad_courses.replace(course, '+'.join(courses))
    for professor, courses in cross_listed_courses.items():
        for course in courses:
            if course in grad_courses.values:
                grad_courses = grad_courses.replace(course, '+'.join(courses))


    # Convert the grad_courses, undergrad_courses dictionary to a list of course names
    grad_courses_list = grad_courses.tolist()

    undergrad_courses_list = [v for k, v in undergrad_courses.items()]

    # Create lists to hold the courses for each year
    freshman_courses = []
    sophomore_courses = []
    junior_courses = []
    senior_courses = []

    # Go through the course list and append to the appropriate list based on the first digit of the course number
    for course in undergrad_courses_list:
        course_parts = course.split(" ")  # Split the course name into parts
        course_number = course_parts[1]  # Get the course number part
        first_digit = course_number[0]  # Get the first digit of the course number

        if first_digit == "1":
            freshman_courses.append(course)
        elif first_digit == "2":
            sophomore_courses.append(course)
        elif first_digit == "3":
            junior_courses.append(course)
        elif first_digit == "4":
            senior_courses.append(course)




    df2 = pd.read_excel(file2.name, sheet_name='availability')

    time_room_available = []

    # rooms = df2.columns[1:]
    # time_slots = df2.iloc[:, 0]
    # for i, time in enumerate(time_slots):
    #     for j, room in enumerate(rooms):
    #         if df2.iloc[i, j+1] == 1:
    #             time_room_available.append((time, room))
    rooms = df2.columns[1:]
    time_slots = df2.iloc[:, 0]
    for i, room in enumerate(time_slots):
        for j, time in enumerate(rooms):
            if df2.iloc[i, j+1] == 1:  # If the value is 1, it means that the classroom is available at this time (Constraints: Class time meets classroom availability)
                time_room_available.append((time, room))

    df_properties = pd.read_excel(file2.name, sheet_name='property')

    classroom_capacity = {}
    classroom_distance = {}

    for i, row in df_properties.iterrows():
        room = row['RoomID']

        classroom_capacity[room] = row['Capacity']
        classroom_distance[room] = row['Distance']




    df_preferences = pd.read_excel(file3.name, sheet_name='time')
    df_preferences.set_index('Time', inplace=True)
    preferences_dict = df_preferences.to_dict()

    for professor, preference in preferences_dict.items():
        for time, value in preference.items():
            if pd.isna(value) or value == ' ':
                preferences_dict[professor][time] = 0
            elif isinstance(value, float):
                preferences_dict[professor][time] = int(value)

    return prof_course_capacity, grad_courses_list, undergrad_courses_list, time_room_available, classroom_capacity, classroom_distance, preferences_dict, pcc_grouped, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict


In [46]:
def process_before_check_excel_files(file4):
    xls = pd.ExcelFile(file4.name)

    all_sheets_prof_course_time_room = []  # This will be a 3D list to hold data from all sheets

    # Iterate over all sheets in the Excel file
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)

        prof_course_time_room = []
        cross_listed_courses = {}

        professors = df.columns[3:27]

        # Undergraduate courses
        undergrad_courses = df.iloc[1:30, 0]
        for i, course in enumerate(undergrad_courses):
            for j, professor in enumerate(professors):
                cell_value = df.iloc[i+1, j+3]
                if cell_value == 1 or cell_value == 'x':
                    time = df.iloc[i+1, 28]
                    room = df.iloc[i+1, 29]
                    prof_course_time_room.append([professor, course, time, room])
                    if cell_value == 'x':
                        if professor not in cross_listed_courses:
                            cross_listed_courses[professor] = [course]
                        else:
                            cross_listed_courses[professor].append(course)

        # Graduate courses
        grad_courses = df.iloc[31:52, 0]
        for i, course in enumerate(grad_courses):
            for j, professor in enumerate(professors):
                cell_value = df.iloc[i+31, j+3]
                if cell_value == 1 or cell_value == 'x':
                    time = df.iloc[i+31, 28]
                    room = df.iloc[i+31, 29]
                    prof_course_time_room.append([professor, course, time, room])
                    if cell_value == 'x':
                        if professor not in cross_listed_courses:
                            cross_listed_courses[professor] = [course]
                        else:
                            cross_listed_courses[professor].append(course)

        # Modify 'prof_course_time_room' to combine cross-listed courses
        for professor, courses in cross_listed_courses.items():
            combined_course = '+'.join(courses)
            for course in courses:
                for i, (prof, old_course, time, room) in enumerate(prof_course_time_room):
                    if prof == professor and old_course == course:
                        prof_course_time_room[i] = [professor, combined_course, time, room]

        # Remove duplicates
        prof_course_time_room = [list(item) for item in set(tuple(row) for row in prof_course_time_room)]

        # Add the data from this sheet to the overall list
        all_sheets_prof_course_time_room.append(prof_course_time_room)
    return all_sheets_prof_course_time_room



In [47]:
def check_schedule(schedules_list, grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available):
    check_grad_course_conflict(schedules_list,grad_courses_list)
    check_class_clash_for_each_level(schedules_list, freshman_courses, 'freshman')
    check_class_clash_for_each_level(schedules_list, sophomore_courses, 'sophomore')
    check_class_clash_for_each_level(schedules_list, junior_courses, 'junior')
    check_class_clash_for_each_level(schedules_list, senior_courses, 'senior')
    check_professor_conflict_schedule(schedules_list)
    check_classroom_capacity(schedules_list, course_capacity_dict, classroom_capacity)
    check_classroom_availability(schedules_list, time_room_available)
    print('----------------------------------------------------------------------------------------------------------------------------------------')
#check：Graduate classes cannot clash

day_codes = ['M', 'T', 'W', 'R', 'F']

def split_timeslots_for_graduate(schedules_list,grad_courses_list):
    split_schedules = []
    for schedule in schedules_list:
        professor, course, timeslot, room = schedule
        if course not in grad_courses_list:  # Ignore if the course is not a graduate course
            continue
        days, time = timeslot.split(' ')
        for day in days:
            split_schedules.append([professor, course, day, time, room])

    # sort the list by professor's name, day of the week (MTWRF), and time
    split_schedules.sort(key=lambda x: (day_codes.index(x[2]), x[3]))
    return split_schedules


def time_to_minutes(time_string):
    # Convert a time in hhmm format to minutes past midnight
    hours = int(time_string[:2])
    minutes = int(time_string[2:])
    return hours * 60 + minutes

def check_grad_course_conflict(schedules_list, grad_courses_list):
    split_schedules = split_timeslots_for_graduate(schedules_list,grad_courses_list)
    for i in range(1, len(split_schedules)):
        prof1, course1, day1, time1, room1 = split_schedules[i - 1]
        prof2, course2, day2, time2, room2 = split_schedules[i]

        # Check if we're looking at schedules for the same graduate course on the same day
        if course1 in grad_courses_list and course2 in grad_courses_list and day1 == day2:
            end_time1 = time_to_minutes(time1.split('-')[1])
            start_time2 = time_to_minutes(time2.split('-')[0])
            # Check if the start time of the second course is before the end time of the first one
            if start_time2 < end_time1:
                print(f"Warning: Graduate courses '{prof1} {course1} {time1}' and '{prof2} {course2} {time2}' have conflict on {day1}.")

def split_timeslots_for_UG(schedules_list, courses_list):
    split_schedules = []
    for schedule in schedules_list:
        professor, course, timeslot, room = schedule
        if course not in courses_list:  # Ignore if the course is not in the level's course list
            continue
        days, time = timeslot.split(' ')
        for day in days:
            split_schedules.append([professor, course, day, time, room])

    # sort the list by day of the week (MTWRF), and time
    split_schedules.sort(key=lambda x: (day_codes.index(x[2]), x[3]))
    return split_schedules

def check_class_clash_for_each_level(schedules_list, courses_list, level_name):
    split_schedules = split_timeslots_for_UG(schedules_list, courses_list)
    for i in range(1, len(split_schedules)):
        prof1, course1, day1, time1, _ = split_schedules[i - 1]
        prof2, course2, day2, time2, _ = split_schedules[i]

        # Check if we're looking at schedules for the same day
        if day1 == day2:
            end_time1 = time_to_minutes(time1.split('-')[1])
            start_time2 = time_to_minutes(time2.split('-')[0])
            # Check if the start time of the second course is before the end time of the first one
            if start_time2 < end_time1:
                print(f"Warning: Undergraduate {level_name} courses '{course1}' and '{course2}' have time clash on {day1}.")

def split_timeslots_for_professor(schedules_list):
    split_schedules = []
    for schedule in schedules_list:
        professor, course, timeslot, room = schedule
        days, time = timeslot.split(' ')
        for day in days:
            split_schedules.append([professor, course, day, time, room])

    # sort the list by professor's name, day of the week (MTWRF), and time
    split_schedules.sort(key=lambda x: (x[0], day_codes.index(x[2]), x[3]))

    return split_schedules

def check_professor_conflict_schedule(schedules_list):
    split_schedules = split_timeslots_for_professor(schedules_list)
    for i in range(1, len(split_schedules)):
        prof1, course1, day1, time1, room1 = split_schedules[i - 1]
        prof2, course2, day2, time2, room2 = split_schedules[i]

        # Check if we're looking at schedules for the same professor on the same day
        if prof1 == prof2 and day1 == day2:
            end_time1 = time_to_minutes(time1.split('-')[1])
            start_time2 = time_to_minutes(time2.split('-')[0])
            # Check if the time difference is less than or equal to 10 minutes
            if start_time2 < end_time1:
                print(f"Error: {prof1} has confilct classes '{course1}' and '{course2}' on {day1}.")
            elif start_time2 - end_time1 <= 10:
                # Check if the rooms are different
                if room1 != room2:
                    print(f"Warning: {prof1} has classes '{course1}' and '{course2}' too close together in different rooms on {day1}.")

def check_classroom_capacity(schedules_list, course_capacity_dict, classroom_capacity):
    for schedule in schedules_list:
        professor, course, time, room = schedule
        course_capacity = course_capacity_dict.get(course)
        if course_capacity is None:
            # Course capacity not specified
            continue

        room_capacity = classroom_capacity.get(room)
        if room_capacity is None:
            print(f"Error: Room {room} capacity not specified for course {course}.")

        if course_capacity > room_capacity:
            print(f"Capacity conflict! Course {course} with {course_capacity} students is scheduled in room {room} with capacity of {room_capacity}.")


def check_classroom_availability(schedules_list, time_room_available):
    for schedule in schedules_list:
        professor, course, time, room = schedule
        if (time, room) not in time_room_available:
            print(f"Error: Professor {professor} is scheduled to teach {course} at {time} in room {room}, which is not available!")

In [48]:


from io import StringIO

def generate_schedule(pcc_grouped, time_room_available, classroom_capacity, preferences_dict, grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses):
    schedule = {}  # Used to store the generated class schedule
    professor_time = {}  # Used to store the time of allocated professors
    grad_courses_times = {}  # Dictionary to store the graduate courses for each time slot
    available_options_list = []  # Create a list to store the number of available time-room options for each course

    #Used to store undergraduate course times
    freshman_times = {}
    sophomore_times = {}
    junior_times = {}
    senior_times = {}

    for professor, courses in pcc_grouped.items():
        for course, capacity in courses:
            # Identify time-classroom combinations that meet capacity needs (Constraints: Classroom capacity meets class size)
            available_tra = [tra for tra in time_room_available if classroom_capacity[tra[1]] >= int(capacity)]
            if professor in professor_time:  # If the professor already has allocated time, it needs to be removed (Constraints: one professor cannot teach at the same time in different rooms)
                available_tra = [tra for tra in available_tra if tra[0] not in professor_time[professor]]
            # Remove the time slots that professor doesn't prefer
            preferred_times = preferences_dict.get(professor, {})
            available_tra = [tra for tra in available_tra if preferred_times.get(tra[0], 1) != 0]
            # Remove the time slots that have graduate courses at this time (Constraints: Graduate classes cannot clash)
            if course in grad_courses_list:
                available_tra = [tra for tra in available_tra if tra[0] not in grad_courses_times]

            # Remove the time slots that have undergraduate courses at this time (Constraints: Same level, UG classes cannot clash)
            if course in freshman_courses:
                available_tra = [tra for tra in available_tra if tra[0] not in freshman_times]
            elif course in sophomore_courses:
                available_tra = [tra for tra in available_tra if tra[0] not in sophomore_times]
            elif course in junior_courses:
                available_tra = [tra for tra in available_tra if tra[0] not in junior_times]
            elif course in senior_courses:
                available_tra = [tra for tra in available_tra if tra[0] not in senior_times]

            # Record the number of available time-room options for this course
            available_options_list.append(len(available_tra))

            # Add a check here: For a given course, based on all constraints, there is no available time-classroom combination
            if not available_tra:
                return None, None  # No valid schedule, exit the function

            selected_tra = random.choice(available_tra)

            # Randomly select one of the time-classroom combinations that meet the constraints
            selected_tra = random.choice(available_tra)
            # 更新课表
            schedule[(professor, course)] = selected_tra
            # 更新教授的时间
            if professor not in professor_time:
                professor_time[professor] = [selected_tra[0]]
            else:
                professor_time[professor].append(selected_tra[0])
            # If this is a graduate course, record the time in grad_courses_times
            if course in grad_courses_list:
                grad_courses_times[selected_tra[0]] = course

            # 根据课程的级别，将所选时间添加到相应的级别时间字典
            if course in freshman_courses:
                freshman_times[selected_tra[0]] = course
            elif course in sophomore_courses:
                sophomore_times[selected_tra[0]] = course
            elif course in junior_courses:
                junior_times[selected_tra[0]] = course
            elif course in senior_courses:
                senior_times[selected_tra[0]] = course

    # create a new list to store the schedules
    schedules_list = []
    for key, value in schedule.items():
        professor, course = key
        time, room = value
        schedules_list.append([professor, course, time, room])
    return schedules_list, available_options_list

def run_schedule_planning(professor_courses_file, prof_course_capacity, grad_courses_list, undergrad_courses_list, time_room_available, classroom_capacity, classroom_distance, preferences_dict, pcc_grouped, freshman_courses, sophomore_courses, junior_courses, senior_courses,course_capacity_dict, iterations, top_n):
    generated_schedules = {}
    weight = 0.5

    for i in range(iterations):
        schedules_list, available_options_list = None, None
        while schedules_list is None or str(schedules_list) in generated_schedules:
            schedules_list, available_options_list = generate_schedule(pcc_grouped, time_room_available, classroom_capacity, preferences_dict, grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses)

        total_distance = 0
        total_preference_score = 0

        for schedule in schedules_list:
            professor, course, time, room = schedule
            preference = int(preferences_dict.get(professor, {}).get(time, 0))
            total_preference_score += preference
            distance = classroom_distance.get(room, 0)
            total_distance += distance

        score = total_preference_score - weight * total_distance
        generated_schedules[str(schedules_list)] = (score, total_preference_score, total_distance)

    sorted_schedules = sorted(generated_schedules.items(), key=lambda x: x[1][0], reverse=True)
    top_n_schedules = sorted_schedules[:top_n]

    def fill_schedule(df, schedule):
        professor, courses, time, room = schedule
        if '+' in courses:
            course_list = courses.split('+')
            for course in course_list:
                fill_schedule(df, [professor, course, time, room])
        else:
            row_number = Courses.eq(courses).idxmax()
            df.loc[row_number, 'Time'] = time
            df.loc[row_number, 'Room'] = room

    # professor_courses_file_data = professor_courses_file.read()
    # df = pd.read_excel(io.BytesIO(professor_courses_file_data), engine='xlrd')
    df = pd.read_excel(professor_courses_file.name)


    Courses = df.iloc[0:52, 0]

    # Initialize an empty list to hold the output
    output_list = []
    output_file_path = '/content/sample_data/schedule plan.xlsx'  # Define the output file path here

    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        for i, schedule in enumerate(top_n_schedules, start=1):
            schedule_str, schedule_info = schedule
            score, total_preference_score, total_distance = schedule_info
            schedule_list = ast.literal_eval(schedule_str)

            # Add these lines to the output list instead of printing
            output_list.append(f"Top {i} schedule in Sheet {i} has score: {score}, total preference score: {total_preference_score}, and the total distance score: {total_distance}")

            # Create a StringIO object to capture check_schedule output
            check_output_io = StringIO()

            # Redirect standard output to check_output_io within this context
            with contextlib.redirect_stdout(check_output_io):
                check_schedule(schedule_list, grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available)

            # Retrieve the captured output as a string and add it to output_list
            check_output = check_output_io.getvalue().strip()
            if check_output:
                output_list.append(f"Checking top {i} schedule:\n{check_output}")

            df_copy = df.copy()

            for schedule in schedule_list:
                fill_schedule(df_copy, schedule)

            df_copy.to_excel(writer, sheet_name=f'Sheet{i}', index=False)

    # Convert the list of output strings to a single string and return it along with the file path
    return "\n".join(output_list), output_file_path


In [49]:
def check_excel_files(all_sheets_prof_course_time_room,grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available):
    i=1
    for schedule in all_sheets_prof_course_time_room:
        print(f"checking sheet {i}:")
        i+=1
        check_schedule(schedule, grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available)
        print("\n")
    return True

In [51]:
# Define global variables
grad_courses_list = None
freshman_courses = None
sophomore_courses = None
junior_courses = None
senior_courses = None
course_capacity_dict = None
classroom_capacity = None
time_room_available = None

def schedule_planner(file1, file2, file3, iterations, top_n):
    # Process the excel files and get the necessary data
    global grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available
    prof_course_capacity, grad_courses_list, undergrad_courses_list, time_room_available, classroom_capacity, classroom_distance, preferences_dict, pcc_grouped, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict = process_excel_files(file1, file2, file3)
    result = run_schedule_planning(file1, prof_course_capacity, grad_courses_list, undergrad_courses_list, time_room_available, classroom_capacity, classroom_distance, preferences_dict, pcc_grouped, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, int(iterations), int(top_n))
    return result[0], result[1]  # Separate the tuple into two parts


def schedule_checker(file4):
    all_sheets_prof_course_time_room=process_before_check_excel_files(file4)
    # Create a StringIO object to capture check_excel_files output
    check_output_io = io.StringIO()
    # Redirect standard output to check_output_io within this context
    with contextlib.redirect_stdout(check_output_io):
        check_excel_files(all_sheets_prof_course_time_room,grad_courses_list, freshman_courses, sophomore_courses, junior_courses, senior_courses, course_capacity_dict, classroom_capacity, time_room_available)
    # Retrieve the captured output as a string
    check_output = check_output_io.getvalue()
    return check_output


with gr.Blocks() as demo:
    with gr.Tab("Schedule Planner"):
        files_input1 = gr.File(label="Professor courses")  # Corrected
        files_input2 = gr.File(label="Classroom property and availability")  # Corrected
        files_input3 = gr.File(label="Professor preference")  # Corrected
        iterations = gr.Number(value=100, label="Iterations")  # Corrected
        top_n = gr.Number(value=10, label="Top N candidates")  # Corrected
        textbox_output1 = gr.Textbox()
        file_output1 = gr.File(label="Schedule plan")
        button1 = gr.Button("Run Planner")

    with gr.Tab("Schedule Checker"):
        file_input4 = gr.File(label="Check schedule plan")  # Corrected
        textbox_output2 = gr.Textbox()
        button2 = gr.Button("Run Checker")

    button1.click(schedule_planner, inputs=[files_input1, files_input2, files_input3, iterations, top_n], outputs=[textbox_output1, file_output1])
    button2.click(schedule_checker, inputs=[file_input4], outputs=[textbox_output2])

demo.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fd87f63f5387176038.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
